In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1045258017237162337, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11140623232
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7342521009400703028
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
!pip install --upgrade tensorflow-gpu==2.4.0
#!pip install --upgrade tensorflow-gpu==2.7.0
#!pip install --upgrade tensorflow-gpu

In [4]:
#@title Import Libraries
# imports
import gc
import tensorflow as tf
from collections import Counter
import numpy as np

import tensorflow as tf

from typing import Callable, Union

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, SpatialDropout2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import max_norm

import time
import os
from skimage.io import imsave, imshow, imsave, imread

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout, Reshape, Permute

from tensorflow.keras.losses import binary_crossentropy


from tensorflow.keras import Model
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import  Sequence


import cv2
from PIL import Image

import re

In [5]:
print(tf.__version__)

2.4.0


In [6]:
#@title Define WormNet

def getWormNet(img_shape=(900,900,1), n_classes=2, pooling='max', img_format='channels_last', dilate_by=1):
    model = Sequential()
    model.add(Conv2D(96, kernel_size=11, strides=6, dilation_rate=dilate_by, activation='relu', padding='same', input_shape=img_shape, data_format=img_format))
    model.add(SpatialDropout2D(0.5))
    model.add(BatchNormalization())
    #model.add(Conv2D(64, kernel_size=5, activation='relu'))
    model.add(MaxPooling2D())
   # model.add(MaxPooling2D(pool_size=2, strides=2))

    
    model.add(Conv2D(256, kernel_size=5, activation='relu'))
    model.add(SpatialDropout2D(0.2))
    model.add(BatchNormalization())
   # model.add(BatchNormalization())
    model.add(MaxPooling2D())
   # model.add(MaxPooling2D(pool_size=2, strides=2))
   # model.add(Dropout(0.2))
    
    model.add(Conv2D(384, kernel_size=3, activation='relu'))
    model.add(SpatialDropout2D(0.2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(384, kernel_size=3, activation='relu'))
    model.add(SpatialDropout2D(0.2))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=3, activation='relu'))
    model.add(SpatialDropout2D(0.2))
    model.add(BatchNormalization())
   # model.add(BatchNormalization())
   
    model.add(MaxPooling2D())
   # model.add(MaxPooling2D(pool_size=2, strides=2))
   # model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(n_classes, activation='softmax'))
    
    return model

In [7]:
# Constants

batchSize = 60 #  60
foldAugment = 30 #30
img_size_factor = 1.0 # 0.75
runningTime = time.strftime('%b-%d-%Y_%H-%M')
numOfClasses = 2
epochs = 100
img_width, img_height = 900, 900
channels = 1
startingLeraningRate = 0.00005 #1e-4, 0.00001
earlyStopFlag = False
modelCheckpointFlag = True
modelContinueFlag = False
reduceLRFlag = True
useRegularizer = False
fineTuneAt = 0
K.set_image_data_format('channels_last')


In [8]:
#@title Define training run

def getClassWeights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cl: float(majority/count) for cl, count in counter.items()}

def one_hot(values):
  n_values = int(np.max(values)) + 1
  return np.eye(n_values)[values]

def countFiles(directory):
    fileCount = 0
    for iDir in os.listdir(directory):
        fileCount += len([name for name in os.listdir(os.path.join(directory,iDir)) if os.path.isfile(os.path.join(directory,iDir, name))])
    return fileCount

def train_run(base_folder, epochs=epochs, model = getWormNet(),
              img_width=img_width, 
              img_height=img_height,
              img_size_factor=img_size_factor,
              foldAugment=foldAugment,
              batchSize=batchSize):
  trainSamplesNumber = countFiles(os.path.join(base_folder,'train'))
  validateSamplesNumber = countFiles(os.path.join(base_folder,'validate'))
  testSamplesNumber = countFiles(os.path.join(base_folder,'test'))

  model.summary()
  model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=startingLeraningRate, 
                             beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy']) # default lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0 or 0.00005

  #class_weights={0:class_weights_array[0],1:class_weights_array[1]}
  
  size = (int(img_width*img_size_factor), int(img_height*img_size_factor))
  
  train_datagen = ImageDataGenerator(
      samplewise_std_normalization=True,
      width_shift_range=0.1,
      height_shift_range=0.1,
      rotation_range=90,
      vertical_flip=True,
      horizontal_flip=True)

  validate_datagen = ImageDataGenerator(
          samplewise_std_normalization=True)

  test_datagen = ImageDataGenerator(
          samplewise_std_normalization=True)

  trainGenerator = train_datagen.flow_from_directory(
      os.path.join(base_folder,'train'),
      color_mode='grayscale',    
      shuffle=True,
      seed=123,
      target_size=size,
      batch_size=batchSize,
      classes=['low','high'],
      class_mode='categorical')

  validationGenerator = validate_datagen.flow_from_directory(
      os.path.join(base_folder,'validate'),
      color_mode='grayscale',
      shuffle=True,
      seed=123,
      target_size=size,
      batch_size=batchSize,
      classes=['low','high'],
      class_mode='categorical')

  testGenerator = test_datagen.flow_from_directory(
      os.path.join(base_folder,'test'),
      color_mode='grayscale',
      shuffle=True,    
      seed=123,
      target_size=size,
      batch_size=batchSize,
      classes=['low','high'],
      class_mode='categorical')


  history = model.fit(
                  trainGenerator,
                  #workers=8,
                  #use_multiprocessing=True,
                  steps_per_epoch=trainSamplesNumber // batchSize,
                  epochs=epochs * foldAugment,
                  verbose=1,
                  validation_data=validationGenerator,
                  class_weight=getClassWeights(trainGenerator.classes),
                  shuffle=True,
                  validation_steps=validateSamplesNumber // batchSize )



  score = model.evaluate_generator(testGenerator, testSamplesNumber)

  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  return history, score

In [9]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
base_folder = '/content/drive/My Drive/worm_models/movement/unseg_imgs/'

history = train_run(base_folder)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 96)      11712     
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 150, 150, 96)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 96)      384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 256)       614656    
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 71, 71, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 71, 71, 256)       1

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 56 images belonging to 2 classes.
Epoch 1/3000
15/15 [==============================] - 346s 21s/step - loss: 2.8360 - accuracy: 0.5137 - val_loss: 2.0685 - val_accuracy: 0.3667
Epoch 2/3000
15/15 [==============================] - 70s 5s/step - loss: 3.6347 - accuracy: 0.5006 - val_loss: 2.6579 - val_accuracy: 0.3833
Epoch 3/3000
15/15 [==============================] - 69s 5s/step - loss: 3.5014 - accuracy: 0.4654 - val_loss: 5.0506 - val_accuracy: 0.3167
Epoch 4/3000
15/15 [==============================] - 70s 5s/step - loss: 2.4577 - accuracy: 0.5010 - val_loss: 7.5352 - val_accuracy: 0.3667
Epoch 5/3000
15/15 [==============================] - 70s 5s/step - loss: 2.1820 - accuracy: 0.5231 - val_loss: 9.5062 - val_accuracy: 0.3167
Epoch 6/3000
15/15 [==============================] - 70s 5s/step - loss: 1.7769 - accuracy: 0.5319 - val_loss: 10.8817 - val_accuracy: 0.3500
Epoch 7/3000
15/15 [==============================] - 69s 5s/step - loss: 1.7068 - accuracy: 0.5226 - val

In [ ]:
import pandas as pd

res = pd.DataFrame.from_dict(history.history)